In [ ]:
# import libs

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_auc_score

In [ ]:
# load and preprocess data
#import os
#os.listdir()

__DATA_DIR__ = "./drive/MyDrive/data-playground/sample_data_final.csv"

df = pd.read_csv(__DATA_DIR__,header = "infer")

df = df.drop(["user_id",
              "orders",
              "room_nights",
              "booking_gbv",
              "orders_outlier_ind",
              "room_nights_outlier_ind",
              "booking_gbv_outlier_ind",
              ],axis=1).reset_index(drop = True)

df.head()

,Unnamed: 0,experiment_variant_code,new_return_visitor,platform_type,posa_super_region,marketing_channel,conv_ind
0,0,VARIANT,RETURN,MOBILE,EMEA,SEM,0
1,1,CONTROL,NEW,MOBILE,APAC,SEM,0
2,3,VARIANT,RETURN,DESKTOP,APAC,META,0
3,4,CONTROL,NEW,MOBILE,EMEA,SEM,0
4,5,VARIANT,NEW,MOBILE,N_AM,DIRECT,0


### Build a classifier to detect if there is bias in assigning groups in features

In [ ]:
train = df[df["experiment_variant_code"]== "CONTROL"].drop("experiment_variant_code",axis = 1)
test = df[df["experiment_variant_code"] == "VARIANT"].drop("experiment_variant_code",axis = 1)

X = train.append(test)
y = [0]*len(train) + [1]*len(test)

In [ ]:
# label encoding

features = ["new_return_visitor","platform_type","posa_super_region","marketing_channel"]

X = pd.get_dummies(X,columns = features)


In [ ]:
# build a simple classifier to check if we can predict chance in the variant set based on attributes

model = RandomForestClassifier()
cv_preds = cross_val_predict(model, X, y, cv=2, n_jobs = None,method = "predict_proba",verbose = 1)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 18.1min finished


In [ ]:
print(roc_auc_score(y_true = y, y_score = cv_preds[:,1]))

0.50027356986075
